<a href="https://colab.research.google.com/github/deepraj16/kaggle-work/blob/main/Understanding_uncoder_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
!pip install -q kaggle


In [ ]:

import kagglehub

# Download latest version
path = kagglehub.dataset_download("umasrikakollu72/hindi-english-truncated-corpus")

print("Path to dataset files:", path)

In [ ]:
df = pd.read_csv("/root/.cache/kagglehub/datasets/umasrikakollu72/hindi-english-truncated-corpus/versions/1")

In [ ]:
import os

path = "/root/.cache/kagglehub/datasets/umasrikakollu72/hindi-english-truncated-corpus/versions/1"
print(os.listdir(path))

In [ ]:
df = pd.read_csv(os.path.join(path, "Hindi_English_Truncated_Corpus.csv"))

In [ ]:
df.head(5)

In [ ]:
lines = df
lines = lines[lines['source'] == 'ted'][['english_sentence', 'hindi_sentence']].dropna().drop_duplicates()
lines = lines.sample(n=25000, random_state=42)


In [ ]:
import string
def clean_text(text):
    exclude = set(string.punctuation)
    text = ''.join(ch for ch in text if ch not in exclude)
    text = text.translate(str.maketrans('', '', string.digits))
    return text.strip().lower()

In [ ]:
lines['english_sentence'] = lines['english_sentence'].apply(clean_text)
lines['hindi_sentence'] = lines['hindi_sentence'].apply(clean_text)
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: 'start_ ' + x + ' _end')

In [ ]:
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(lines['english_sentence'])
eng_seq = eng_tokenizer.texts_to_sequences(lines['english_sentence'])

hin_tokenizer = Tokenizer(filters='')
hin_tokenizer.fit_on_texts(lines['hindi_sentence'])
hin_seq = hin_tokenizer.texts_to_sequences(lines['hindi_sentence'])

In [ ]:
max_eng_len = max(len(seq) for seq in eng_seq)
max_hin_len = max(len(seq) for seq in hin_seq)

encoder_input = pad_sequences(eng_seq, maxlen=max_eng_len, padding='post')
decoder_input = pad_sequences(hin_seq, maxlen=max_hin_len, padding='post')

In [ ]:
decoder_target = np.zeros((decoder_input.shape[0], decoder_input.shape[1], 1))
decoder_target[:, 0:-1, 0] = decoder_input[:, 1:]

In [ ]:
from tensorflow.keras.layers import Input, Embedding, LSTM
from tensorflow.keras.models import Model

eng_vocab_size = 25000
latent_dim = 20


In [ ]:
from tensorflow.keras.layers import Input, Embedding, LSTM
from tensorflow.keras.models import Model

# Define input layer
encoder_inputs = Input(shape=(None,), name="encoder_input")

# Embedding layer
enc_emb = Embedding(input_dim=eng_vocab_size, output_dim=latent_dim, name="encoder_embedding")(encoder_inputs)

# LSTM layer
enc_outputs, state_h, state_c = LSTM(latent_dim, return_state=True, name="encoder_lstm")(enc_emb)

# Encoder states to be passed to the decoder
encoder_states = [state_h, state_c]


In [ ]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

# Decoder input
decoder_inputs = Input(shape=(None,), name="decoder_input")
hin_vocab_size =25000   # 🔁 Set this to your actual Hindi vocabulary size
latent_dim = 30        # 🔁 Match with encoder's latent_dim
encoder_states = [state_h, state_c]  # from encoder LSTM

dec_emb_layer = Embedding(input_dim=hin_vocab_size, output_dim=latent_dim, name="decoder_embedding")
dec_emb = dec_emb_layer(decoder_inputs)

# LSTM with encoder states as initial state
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name="decoder_lstm")
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

# Dense softmax layer to predict output tokens
decoder_dense = Dense(hin_vocab_size, activation='softmax', name="decoder_output")
decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:

model.fit(
    [encoder_input, decoder_input],
    decoder_target,
    batch_size=64,
    epochs=15,
    validation_split=0.2
)


In [ ]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model

latent_dim = 256
eng_vocab_size = 10000
hin_vocab_size = 10000

# Encoder
encoder_inputs = Input(shape=(None,), name='encoder_input')
enc_emb = Embedding(input_dim=eng_vocab_size, output_dim=latent_dim)(encoder_inputs)
_, state_h, state_c = LSTM(latent_dim, return_state=True)(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,), name='decoder_input')
dec_emb = Embedding(input_dim=hin_vocab_size, output_dim=latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
print("Before Dense:", decoder_outputs.shape)  # (batch_size, seq_len, latent_dim)

decoder_dense = Dense(hin_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
print("After Dense:", decoder_outputs.shape)  # (batch_size, seq_len, hin_vocab_size)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
